In [1]:
import numpy as np
import tensorflow as tf
from datetime import datetime

In [2]:
SERVER0 = '/device:CPU:0'
SERVER1 = '/device:CPU:1'
CRYPTO_PRODUCER = '/device:CPU:2'
INPUT_PROVIDER  = '/device:CPU:3'
OUTPUT_RECEIVER = '/device:CPU:4'

config = tf.ConfigProto(
    log_device_placement=True,
    device_count={"CPU": 5},
    inter_op_parallelism_threads=1,
    intra_op_parallelism_threads=1
)

In [3]:
Q = 10001

def share(secret):
    share0 = tf.random_uniform(secret.shape, maxval=Q, dtype=tf.int64)
    share1 = (secret - share0) % Q
    return share0, share1

def reconstruct(share0, share1):
    return (share0 + share1) % Q

def sample(shape):
    return tf.random_uniform(shape, maxval=Q, dtype=tf.int64)

In [4]:
def add(x, y):
    
    x0, x1 = x
    y0, y1 = y
    
    with tf.device(SERVER0):
        z0 = (x0 + y0) % Q

    with tf.device(SERVER1):
        z1 = (x1 + y1) % Q
        
    return z0, z1

In [5]:
def sub(x, y):
    
    x0, x1 = x
    y0, y1 = y
    
    with tf.device(SERVER0):
        z0 = (x0 - y0) % Q

    with tf.device(SERVER1):
        z1 = (x1 - y1) % Q
        
    return z0, z1

In [6]:
def mul(x, y):
    
    x0, x1 = x
    y0, y1 = y

    with tf.device(CRYPTO_PRODUCER):
        a = sample((10,10))
        b = sample((10,10))
        ab = (a * b) % Q

        a0, a1 = share(a)
        b0, b1 = share(b)
        ab0, ab1 = share(ab)

    with tf.device(SERVER0):
        alpha0 = (x0 - a0) % Q
        beta0  = (y0 - b0) % Q

    with tf.device(SERVER1):
        alpha1 = (x1 - a1) % Q
        beta1  = (y1 - b1) % Q

    # exchange of alpha's and beta's
        
    with tf.device(SERVER0):
        alpha = reconstruct(alpha0, alpha1)
        beta = reconstruct(beta0, beta1)
        z0 = ( ab0 + (a0 * beta) + (b0 * alpha) + (alpha * beta) ) % Q

    with tf.device(SERVER1):
        alpha = reconstruct(alpha0, alpha1)
        beta = reconstruct(beta0, beta1)
        z1 = ( ab1 + (a1 * beta) + (b1 * alpha) ) % Q
        
    return z0, z1

In [7]:
def define(shape):
    
    with tf.device(INPUT_PROVIDER):
        m = tf.placeholder(tf.int64, shape=(10,10))
        x = share(m)
        
    return m, x

In [8]:
def variable(shape):
    
    with tf.device(SERVER0):
        x0 = tf.Variable(tf.ones(shape=shape, dtype=tf.int64))

    with tf.device(SERVER1):
        x1 = tf.Variable(tf.ones(shape=shape, dtype=tf.int64))
        
    return x0, x1

In [9]:
# def assign(x):
    
#     x0, x1 = x
    
    
    

In [10]:
def reveal(x):
    
    x0, x1 = x

    with tf.device(OUTPUT_RECEIVER):
        y = reconstruct(x0, x1)
    
    return y

In [11]:
input_x, x = define((10,10))

w = variable((10,10))
b = variable((10,10))

y_pred = add(mul(x, w), b)

# error = sub(y_pred, y)
# gradients = 2/m * tf.matmul(tf.transpose(X), error)
# training_op = tf.assign(theta, theta - learning_rate * gradients)

p = reveal(y_pred)

In [12]:
with tf.Session(config=config) as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    
    res = sess.run([p], {
        input_x: np.arange(10*10).reshape(10,10).astype(int),
    })
    print(res)

[array([[  2,   4,   6,   8,  10,  12,  14,  16,  18,  20],
       [ 22,  24,  26,  28,  30,  32,  34,  36,  38,  40],
       [ 42,  44,  46,  48,  50,  52,  54,  56,  58,  60],
       [ 62,  64,  66,  68,  70,  72,  74,  76,  78,  80],
       [ 82,  84,  86,  88,  90,  92,  94,  96,  98, 100],
       [102, 104, 106, 108, 110, 112, 114, 116, 118, 120],
       [122, 124, 126, 128, 130, 132, 134, 136, 138, 140],
       [142, 144, 146, 148, 150, 152, 154, 156, 158, 160],
       [162, 164, 166, 168, 170, 172, 174, 176, 178, 180],
       [182, 184, 186, 188, 190, 192, 194, 196, 198, 200]])]
